In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
import os
import numpy as np

In [2]:
# Directories
train_dir = '/Users/lalitaneeharikavajjhala/Desktop/Images_bottles/'

# Check if directories exist
if not os.path.exists(train_dir):
    print(f"Directory {train_dir} does not exist.")
else:
    defective_dir = os.path.join(train_dir, 'defective_bottles')
    non_defective_dir = os.path.join(train_dir, 'non_defective_bottles')
    
    if not os.path.exists(defective_dir):
        print(f"Directory {defective_dir} does not exist.")
    if not os.path.exists(non_defective_dir):
        print(f"Directory {non_defective_dir} does not exist.")


In [3]:
# Image Data Generator
train_datagen = ImageDataGenerator(
    rescale=1./255,        #rescale the pixels to the range [0-1] from the range [0-255]
    validation_split=0.2,  # 20% validation split
)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(150, 150),  # Resize images to 150x150 pixels
    batch_size=32,           #number of images to be processed per batch
    class_mode='binary',    # Binary classification
    subset='training'       #makes the image data generator focus on just the training data
)

validation_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='binary',
    subset='validation'
)

Found 19 images belonging to 2 classes.
Found 4 images belonging to 2 classes.


In [4]:
# Debug prints
print(f"Classes found: {train_generator.class_indices}")
print(f"Number of training samples: {train_generator.samples}")
print(f"Number of validation samples: {validation_generator.samples}")

# CNN Model
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)),      #initial layer, 32 filters, (3x3) is the std kernel size, relu activation introduces non linearity, which helps the model understand complex patterns
    MaxPooling2D((2, 2)),                        #maxpooling layer reduces the dimensional complexities and helps reduce overfitting, with pooling size 2x2, which ensures enough info is gathered 
    Conv2D(64, (3, 3), activation='relu'),                #input shape corresponds to the 150x150 pixels and 3 colors - RGB
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),                         #helps convert the 2D array to a 1D array suitable for the Dense layers
    Dense(512, activation='relu'),      #the dense layers do the actual classification 
    Dropout(0.5),                       #dropout layer ignores unnecessary info preventing overfitting with a dropout rate of 50%
    Dense(1, activation='sigmoid')      #the sigmoid function helps in getting an output that is the probability, in the range 0-1
])

# Compile the model
model.compile(
    optimizer=Adam(),            #adam optimizer sets and updates the learning rates for each parameter individually, according to the properties of the parameters
    loss='binary_crossentropy',    #prevents the loss function value
    metrics=['accuracy']
)

Classes found: {'defective_bottles': 0, 'non_defective_bottles': 1}
Number of training samples: 19
Number of validation samples: 4


In [5]:
# Train the model only if there are samples
if train_generator.samples > 0 and validation_generator.samples > 0:     #checks whether samples are present in the training and validation sets
    history = model.fit(            #performs validation 
        train_generator,        #the generator that provides batches of trained data
        steps_per_epoch=max(1, train_generator.samples // train_generator.batch_size),     #number of batches of samples to train the model on per epoch, ensuring model trains on the entire dataset (value = 1)
        validation_data=validation_generator,
        validation_steps=max(1, validation_generator.samples // validation_generator.batch_size),
        epochs=10     #number of epochs that the validation process will run for 
    )
    
    # Save the model to a writable directory
    model_save_path = os.path.expanduser('~/bottle_classifier.h5')
    model.save(model_save_path)
    print(f"Model saved to {model_save_path}")

    # Print the accuracy
    training_accuracy = history.history['accuracy'][-1]
    validation_accuracy = history.history['val_accuracy'][-1]
    print(f"Training Accuracy: {training_accuracy*100:.4f}")
    print(f"Validation Accuracy: {validation_accuracy*100:.4f}")

else:
    print("No training or validation samples found. Please check the dataset directory and structure.")


Epoch 1/10
1/1 [==============================] - 1s 877ms/step - loss: 0.6826 - accuracy: 0.5263 - val_loss: 2.3438 - val_accuracy: 0.5000
Epoch 2/10
1/1 [==============================] - 0s 439ms/step - loss: 1.9877 - accuracy: 0.5789 - val_loss: 1.7394 - val_accuracy: 0.5000
Epoch 3/10
1/1 [==============================] - 0s 446ms/step - loss: 2.1376 - accuracy: 0.4211 - val_loss: 0.7966 - val_accuracy: 0.5000
Epoch 4/10
1/1 [==============================] - 0s 439ms/step - loss: 0.9435 - accuracy: 0.4211 - val_loss: 0.7403 - val_accuracy: 0.5000
Epoch 5/10
1/1 [==============================] - 0s 437ms/step - loss: 0.6306 - accuracy: 0.5789 - val_loss: 0.7736 - val_accuracy: 0.5000
Epoch 6/10
1/1 [==============================] - 0s 438ms/step - loss: 0.6002 - accuracy: 0.6842 - val_loss: 0.7404 - val_accuracy: 0.5000
Epoch 7/10
1/1 [==============================] - 0s 436ms/step - loss: 0.5672 - accuracy: 0.8421 - val_loss: 0.8412 - val_accuracy: 0.5000
Epoch 8/10
1/1 [====

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [6]:
def predict_defect(image_path):
    # Load the saved model
    model = load_model(os.path.expanduser('/Users/lalitaneeharikavajjhala/bottle_classifier.h5'))
    
    # Load and preprocess the image
    img = load_img(image_path, target_size=(150, 150))   #resizing the image to match the initial dimensions 
    img_array = img_to_array(img)      #converts image from python image library format to numpy array format
    img_array = np.expand_dims(img_array, axis=0)     #adds a dimension that makes it suitable to be processed by the model
    img_array /= 255.0  # Rescale image to [0-1] pixels
    
    # Predict the class
    prediction = model.predict(img_array)
    
    if prediction[0] > 0.5:
        print(f"The bottle in the image is predicted to be defective.")
    else:
        print(f"The bottle in the image is predicted to be non-defective.")

In [7]:
# Example usage
predict_defect('/Users/lalitaneeharikavajjhala/Desktop/Images_bottles/image_test.jpg')

1/1 [==============================] - 0s 44ms/step
The bottle in the image is predicted to be non-defective.
